In [ ]:
#Importer les packages necessaires

import pandas as pd       # pandas : pour la manipulation et l’analyse de données tabulaires (DataFrame)
import numpy as np        # numpy : pour les calculs numériques et les tableaux multidimensionnels
import matplotlib.pyplot as plt  # matplotlib.pyplot : pour créer des graphiques et visualiser les données
import seaborn as sns     # seaborn : pour des visualisations statistiques plus esthétiques et plus simples que matplotlib
from scipy import stats    # scipy.stats : pour les tests statistiques et les fonctions de distribution
import statsmodels.stats.multitest as smm  # statsmodels.stats.multitest : pour corriger les tests multiples (ex. méthode de Benjamini-Hochberg)

In [ ]:
# Charger le jeu de données
df = pd.read_csv("toy_proteomics_tumor_necrosis_healthy.csv")
df.head()

In [ ]:
# compte le nombre d'échantillons dans la colonne 'Class'
df['Class'].value_counts()  

In [ ]:
#histogramme de Class
# figure avec une taille de 6 pouces sur 5
plt.figure(figsize=(6,5))  

# Diagramme en barres (countplot) affichant le nombre d’échantillons la colonne 'Class'.
# 'palette="Set2"' donne une palette de couleurs douce et harmonieuse pour les barres. vous pouvez personnaliser les couleurs et chnager de palette (voir doc seabron)

sns.countplot(data=df, x="Class", palette="Set2")

#un titre au graphique
plt.title("Répartition des échantillons par classe")  

#un nom à l’axe des abscisses (x).
plt.xlabel("Classe")  

# un nom à l’axe des ordonnées (y).
plt.ylabel("Nombre d'échantillons")

# Ajuste automatiquement la disposition pour éviter que les titres/étiquettes se chevauchent.
plt.tight_layout()  

# Affiche le graphique à l’écran
plt.show()  

In [ ]:
# Statistiques descriptives
df.describe()

In [ ]:
#Verifier s'il y des valeures manquantes ?
#supprimer les protéines avec les valeurs manquantes ou imputer avec "median"

In [ ]:
#Vérification de la normalité d’une variable avec un QQ-plot

protein = "EGFR"                  # Nom de la protéines (ici : 'EGFR')
data = df[protein].dropna()       # Extrait les valeurs de la colonne 'EGFR' en supprimant les valeurs manquantes (NaN)

# QQ-plot (Quantile-Quantile plot)
stats.probplot(data, dist="norm", plot=plt)  # Génère un QQ-plot comparant la distribution des données à une distribution normale théorique
plt.title(f"QQ-plot pour {protein}")         # un titre dynamique au graphique, indiquant la variable analysée
plt.show()                                   # Affiche le graphique à l’écran

In [ ]:
#Vérification de la normalité par classe (Tumor vs Healthy...)

# QQ-plot

# Sélectionne uniquement les valeurs de la protéine pour les échantillons appartenant à la classe "Tumor"
# et supprime les valeurs manquantes (NaN)
data_tumor = df[df['Class'] == 'Tumor'][protein].dropna()    

# Sélectionne les valeurs de la protéine pour la classe "Healthy"
# et supprime également les valeurs manquantes
data_healthy = df[df['Class'] == 'Healthy'][protein].dropna()  


# Crée une figure avec 1 ligne et 2 sous-graphiques (axes) côte à côte
# La taille totale du graphique est de 10 pouces sur 4
fig, axes = plt.subplots(1, 2, figsize=(10,4))  

# Trace le QQ-plot pour les échantillons "Tumor", comparant leur distribution à une distribution normale
stats.probplot(data_tumor, dist="norm", plot=axes[0])
# Définit le titre du premier graphique
axes[0].set_title(f"{protein} - Tumor") 


# Trace le QQ-plot pour les échantillons "Healthy"
stats.probplot(data_healthy, dist="norm", plot=axes[1])  
# Définit le titre du deuxième graphique
axes[1].set_title(f"{protein} - Healthy")  

# Affiche les deux QQ-plots côte à côte
plt.show()  

In [ ]:
#Test de normalité de Shapiro-Wilk (example sur les échantillons tumoraux) 

# Applique le test de Shapiro-Wilk sur les données tumorales (data_tumor, varaibles créee précedemment)
# 'stat' correspond à la statistique W du test
# 'p' est la valeur p associée au test (probabilité que les données suivent une loi normale)
stat, p = stats.shapiro(data_tumor)  

# Résultats du test avec 3 décimales : la statistique W et la p-valeur
print(f"Shapiro-Wilk test : W={stat:.3f}, p={p:.3f}")  

if p > 0.05:
    print("Distribution normale (p>0.05)")  
    # Si p > 0.05, on ne rejette pas l’hypothèse nulle (H0) → la distribution est considérée comme normale
else:
    print("Distribution non normale (p<0.05)")  
    # Si p < 0.05, on rejette l’hypothèse nulle → la distribution n’est pas normale

In [ ]:
#Test t de Student entre les groupes “Tumor” et “Healthy”

# t-test entre Cancer et Healthy

protein = "EGFR"  

# Sélectionne les valeurs de la protéine 'EGFR' pour les échantillons appartenant à la classe "Tumor"
group1 = df[df['Class'] == 'Tumor'][protein]  

# Sélectionne les valeurs de la même protéine pour les échantillons de la classe "Healthy"
group2 = df[df['Class'] == 'Healthy'][protein]  

# Test t de Student
# Réalise un test t de Student pour échantillons indépendants entre les deux groupes
# 'equal_var=False' applique la version de Welch du test t, qui ne suppose pas d’égalité de variances
# t_stat = valeur de la statistique t
# p_val = p-valeur associée au test
t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=False)  

# Affiche le résultat du test : la statistique t et la p-valeur (notation scientifique pour plus de lisibilité)
print(f"T-test {protein} : t={t_stat:.3f}, p={p_val:.3e}")  


In [ ]:
#Test t pour chaque protéine et correction de faux postifs (FDR – Benjamini-Hochberg)

# Initialise une liste vide pour stocker les p-valeurs obtenues pour chaque protéine
pvals = []  

for prot in df.columns[1:]:  
    # Boucle sur toutes les colonnes du DataFrame à partir de la 2e (en supposant que la 1ère est 'Class')
    
    g1 = df[df['Class'] == 'Tumor'][prot]  
    # Sélectionne les valeurs de la protéine actuelle pour les échantillons du groupe "Tumor"
    
    g2 = df[df['Class'] == 'Healthy'][prot]  
    # Sélectionne les valeurs de la même protéine pour le groupe "Healthy"
    
    _, p = stats.ttest_ind(g1, g2, equal_var=False)  
    # Réalise un test t de Student (version de Welch) entre les deux groupes pour la protéine en cours
    # Retourne la p-valeur associée à la comparaison
    
    pvals.append(p)  
    # Ajoute la p-valeur obtenue à la liste des p-valeurs

# Correction Benjamini-Hochberg (FDR)
# Applique une correction de Benjamini-Hochberg pour contrôler le taux de fausses découvertes (FDR)
# 'pvals_corr' = p-valeurs ajustées
# 'rejected' = tableau booléen indiquant si l’hypothèse nulle est rejetée (True si significatif après correction)

rejected, pvals_corr, _, _ = smm.multipletests(pvals, alpha=0.05, method='fdr_bh')  

results = pd.DataFrame({
    "Protein": df.columns[1:],     # Nom de chaque protéine testée
    "p_value": pvals,              # P-valeur brute du test t
    "p_adj": pvals_corr,           # P-valeur ajustée après correction FDR
    "Significant": rejected        # Indique si la différence est significative après correction
})
results

In [ ]:
# Boxplot pour une protéine
proteine =["EGFR"]
sns.boxplot(x="Class", y=protein, data=df)
plt.title(f"Expression de {protein}")
plt.show()

In [ ]:
# swarmplot pour une protéine
proteine =["EGFR"]
sns.swarmplot(x="Class", y=protein, data=df, color=".25")
plt.title(f"Expression de {protein}")
plt.show()

In [ ]:
# Violinplot combiné avec swarmplot pour une protéine
proteine =["EGFR"]
sns.violinplot(x="Class", y=protein, data=df)
sns.swarmplot(x="Class", y=protein, data=df, color=".25")
plt.title(f"Expression de {protein}")
plt.show()

In [ ]:
#barplot avec sd

protein = "EGFR"
sns.barplot(data=df, x="Class", y=protein, estimator=np.mean, errorbar="sd", hue="Class", palette="Set2")
plt.title(f"Barplot — moyenne de {protein} par classe")
plt.xlabel("Classe")
plt.ylabel(f"Moyenne de {protein}")
plt.tight_layout()
plt.show()

In [ ]:
#Visualisation (boxplot, violinplot...) d'expression d’une protéine avec test statistique et annotations de p-values

from itertools import combinations    # Pour générer automatiquement toutes les combinaisons de paires de classes
from statannotations.Annotator import Annotator  # Pour ajouter des annotations de tests statistiques sur les graphiques

# Choisir la protéine et le test à utiliser
protein = "EGFR"      
test_choice = "Mann-Whitney"   # Choix du test statistique à appliquer entre les groupes (ex : t-test_ind, Mann-Whitney, etc.)


# Créer le graphique de base
plt.figure(figsize=(8,6))  
# Définit la taille du graphique

ax = sns.violinplot(x="Class", y=protein, data=df, inner="box", palette="Set2")  
# Crée un violin plot montrant la distribution de la protéine selon la classe (Tumor vs Healthy)
# 'inner="box"' ajoute une boîte à moustaches à l’intérieur pour représenter la médiane et les quartiles

sns.swarmplot(x="Class", y=protein, data=df, color=".25", size=4)  
# Ajoute un swarm plot (points individuels) par-dessus le violon plot pour montrer chaque valeur observée

# Déterminer les comparaisons par paires automatiquement
classes = df["Class"].dropna().unique().tolist()  
# Récupère la liste des classes présentes dans la colonne 'Class' (sans valeurs manquantes)

classes.sort()  
# Trie les classes par ordre alphabétique pour une présentation cohérente

pairs = list(combinations(classes, 2))   
# Crée la liste de toutes les comparaisons possibles entre les classes (ex. [('Healthy', 'Tumor')])


# Ajouter les annotations (p-values sous forme d’étoiles)
annotator = Annotator(ax, pairs, data=df, x="Class", y=protein)  
# Initialise l’objet Annotator en liant le graphique, les paires de comparaison, et les données

annotator.configure(test=test_choice, text_format='star', loc='inside', verbose=0)  
# Configure le test statistique choisi (ici Mann-Whitney)
# 'text_format="star"' affiche la significativité avec des étoiles (*, **, ***)
# 'loc="inside"' place les annotations à l’intérieur du graphique
# 'verbose=0' évite l’affichage des détails dans la console

annotator.apply_and_annotate()  
# Applique le test statistique sur chaque paire et ajoute les annotations sur le graphique


# Mise en forme
plt.title(f"Expression de {protein} — Test : {test_choice}")  
# Ajoute un titre au graphique indiquant la protéine et le test utilisé

plt.xlabel("Classe")  
# Nom de l’axe des x

plt.ylabel(f"Niveau d’expression de {protein}")  
# Nom de l’axe des y

plt.tight_layout()  


plt.show()  



In [ ]:
# Analyse de corrélation entre deux protéines avec scatter plot et régression

# Exemple : corrélation entre deux protéines
x_var = "EGFR"   # Protéine à placer sur l'axe des x
y_var = "MKI67"  # Protéine à placer sur l'axe des y

# Crée une figure de 6 pouces sur 5 pour le graphique
plt.figure(figsize=(6,5))  

sns.scatterplot(data=df, x=x_var, y=y_var, hue="Class", palette="Set2", s=60, alpha=0.8)  
# Trace un nuage de points (scatter plot)
# 'hue="Class"' colore les points selon la classe (Tumor vs Healthy...)
# 's=60' définit la taille des points, 'alpha=0.8' rend les points légèrement transparents

sns.regplot(data=df, x=x_var, y=y_var, scatter=False, color='black', ci=None)  
# Trace la ligne de régression linéaire sur le scatter plot
# 'scatter=False' empêche l’affichage des points supplémentaires
# 'ci=None' supprime la zone de confiance autour de la ligne


plt.title(f"Scatter Plot — {x_var} vs {y_var}")  

plt.xlabel(x_var)  

plt.ylabel(y_var)  

plt.legend(title="Class")  

plt.tight_layout()  

plt.show()  


# Calcul de la corrélation Pearson
from scipy.stats import pearsonr  # ou spearmanr
r, p = pearsonr(df[x_var], df[y_var])  
# Calcule le coefficient de corrélation de Pearson entre les deux variables
# r = coefficient de corrélation (-1 à 1)
# p = p-valeur associée à la corrélation

In [ ]:
#Matrice de corrélation

# Sélectionner uniquement les colonnes numériques
data_num = df.select_dtypes(include=[np.number]) 

# Calcul de la matrice de corrélation (Spearman)
corr_matrix = data_num.corr(method='spearman') 

plt.figure(figsize=(10,8)) 

sns.heatmap(corr_matrix, cmap="coolwarm", annot=False, center=0, linewidths=0.5)  
# Trace la matrice de corrélation sous forme de heatmap
# 'cmap="coolwarm"' définit la palette de couleurs (bleu pour négatif, rouge pour positif)
# 'annot=False' n’affiche pas les valeurs sur la heatmap (peut mettre True si besoin)
# 'center=0' centre la palette autour de 0
# 'linewidths=0.5' ajoute de fines lignes entre les cases pour la lisibilité


plt.title("Matrice de corrélation (Spearman)")

plt.tight_layout()  

plt.show()  


In [ ]:
# volcano plot

# Classes à comparer
class1 = "Tumor"
class2 = "Healthy"

# Seuils pour significance
p_threshold = 0.5
fc_threshold = 0  # correspond à |log2FC| ≥ 1 (≈ FC ≥ 2 ou ≤ 0.5)

# ==============================================================
# ANALYSE DIFFÉRENTIELLE (t-test + log2FC + FDR)
# ==============================================================

proteins = df.columns.drop("Class")
results = pd.DataFrame(index=proteins, columns=["p_value", "log2FC"])

# Calcul p-value et log2FC pour chaque protéine
for prot in proteins:
    group1 = df[df["Class"] == class1][prot].dropna()
    group2 = df[df["Class"] == class2][prot].dropna()
    
    # T-test (paramétrique, tu peux remplacer par Mann–Whitney si besoin)
    _, p = stats.ttest_ind(group1, group2, equal_var=False)
    results.loc[prot, "p_value"] = p
    
    # Log2 Fold Change
    mean1 = np.mean(group1)
    mean2 = np.mean(group2)
    results.loc[prot, "log2FC"] = np.log2((mean1 + 1e-9) / (mean2 + 1e-9))  # éviter div/0

# Correction FDR (Benjamini–Hochberg)
results["p_value"] = results["p_value"].astype(float)
_, results["p_adj"], _, _ = smm.multipletests(results["p_value"], alpha=0.05, method='fdr_bh')

# ==============================================================
# CATÉGORISATION (UP / DOWN / NS)
# ==============================================================

def classify(row):
    if row["p_adj"] < p_threshold and row["log2FC"] >= fc_threshold:
        return "Upregulated"
    elif row["p_adj"] < p_threshold and row["log2FC"] <= -fc_threshold:
        return "Downregulated"
    else:
        return "Not significant"

results["Regulation"] = results.apply(classify, axis=1)

# ==============================================================
# VOLCANO PLOT
# ==============================================================

plt.figure(figsize=(8,6))

sns.scatterplot(
    data=results,
    x="log2FC",
    y=-np.log10(results["p_adj"]),   #
    hue="Regulation",
    palette={"Upregulated": "red", "Downregulated": "blue", "Not significant": "grey"},
    alpha=0.8,
    edgecolor=None
)
top_hits = results[results["Regulation"] != "Not significant"].sort_values("p_adj").head(10)
for prot in top_hits.index:
    x = results.loc[prot, "log2FC"]
    y = -np.log10(results.loc[prot, "p_adj"])
    plt.text(x, y, prot, fontsize=8, ha='right')

# Seuils
plt.axhline(-np.log10(p_threshold), color='black', linestyle='--', linewidth=1)
plt.axvline(fc_threshold, color='black', linestyle='--', linewidth=1)
plt.axvline(-fc_threshold, color='black', linestyle='--', linewidth=1)

# Mise en forme

plt.title(f"Volcano Plot — {class1} vs {class2} (p_adj)")
plt.xlabel("log2(Fold Change)")
plt.ylabel("-log10(p_adj)")

plt.legend(title="Regulation", loc="upper left")
plt.tight_layout()
plt.show()
